<a href="https://colab.research.google.com/github/mattiss/M3uJuliaParser/blob/main/julia_prepare_m3u_playlists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.2 on the current Colab Runtime...
2022-04-15 18:08:16 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz [123295596/123295596] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...


# Reading the original m3u file
Change that to the relevant directory

In [3]:
m3u_file = "/home/jovyan/work/dev/julia/data/m3u/tv_channels_SWFAF06091.m3u"
m3u_file = "http://mvu03.wanicelife.com:8880/get.php?username=SWFAF06091&password=BUPY5CDE&type=m3u&output=mpegts"

"http://mvu03.wanicelife.com:8880/get.php?username=SWFAF06091&password=BUPY5CDE&type=m3u&output=mpegts"

In [4]:
const REGEX_M3U_FILE_HEADER = r"^#EXTM3U$"
const REGEX_M3U_GROUP_SEPARATOR = r"^#EXTINF:-1,(?<name>((?<country>.*):)?-*\|(?<group>.*)\|-*)"
const REGEX_M3U_CHANNEL_INFORMATION = r"^#EXTINF:-1,((?<id>.*):)?(?<name>.*?)( (?<res>(SD)|(HD)|(FHD)))?$"

channels = []
open(m3u_file) do file
    header = readline(file)
    if (!occursin(REGEX_M3U_FILE_HEADER, header))
        println("File $(m3u_file) is not a valid m3u file")
    end
    i = 0
    global country = ""
    global group = ""
    global id = ""
    global name = ""
    global res = ""
    global url = ""
    for ln in eachline(file)
        r = match(REGEX_M3U_GROUP_SEPARATOR, ln)
        if (!isnothing(r))
            #println("Processing Group $(r["country"]):$(r["group"])...")
            country, group = r["country"], r["group"]
        end
        r = match(REGEX_M3U_CHANNEL_INFORMATION, ln)
        if (!isnothing(r))
            id, name, res = r["id"], r["name"], r["res"]
        else
            channel = (country = country, group = group, id = id, name = name, res = res, url=ln)
            push!(channels, channel)
        end
    end
end
println("Found $(length(channels)) channels")

LoadError: ignored

In [ ]:
using DataFrames
df = DataFrame(channels)
first(df,5)

Sun Jan  9 02:26:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
const M3U_FILE_HEADER = "#EXTM3U"

function print_channel(name, group, url)
    println("#EXTINF:-1 group-title=\"$group\",$name $res")
    println(url)
end

function print_df(df)
    s = ""
    println(M3U_FILE_HEADER)
    for row in eachrow(df)
        print_channel(row["name"], row["group"], row["url"])   
    end
end

#print_df(df_fr)

In [ ]:
function write_channel(f, name, group, url)
    println(f, "#EXTINF:-1 group-title=\"$group\",$name")
    println(f, url)
end

function write_df(output_file, df)
    open(m3u_output_file, "w") do file
        println(file, M3U_FILE_HEADER)
        for row in eachrow(df)
            if !isnothing(row["res"])
                write_channel(file, row["name"]*" "*row["res"], row["group"], row["url"])  
            else
                write_channel(file, row["name"], row["group"], row["url"])  
            end
        end
    end
end

In [ ]:
m3u_output_file = "/home/jovyan/work/dev/julia/data/m3u/playlist.m3u"
df_fr = filter(row -> row.country == "FR", df)
write_df(m3u_output_file, df_fr)

In [ ]:
m3u_output_file = "/home/jovyan/work/dev/julia/data/m3u/playlist.usa.m3u"
df_usa = filter(row -> row.country == "USA", df)
write_df(m3u_output_file, df_usa)

In [ ]:
m3u_output_file = "/home/jovyan/work/dev/julia/data/m3u/playlist.xxx.m3u"
df_xxx = filter(row -> row.group == "xxx-VIP", df)
write_df(m3u_output_file, df_xxx)

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />